# Creating a single record for each patient across clinical, genomic, and imaging datasets

In [15]:
import pandas
import boto3
import time

In [16]:
from sagemaker import get_execution_role

In [17]:
#Initializing the bucket where data will be stored
# Using the in built Amazon SageMaker execution role

role = get_execution_role()
bucket='multimodal-dataset-clinical-genomic-imaging'
data_key = 'single-patient-records'
data_location = 's3://{}/{}'.format(bucket, data_key)

In [18]:
# Installing pyathena to connect to athena to run queries on the notebook
!pip install pyathena


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [19]:
from pyathena import connect
conn = connect(s3_staging_dir=data_location,
               region_name='us-east-1')

## Clinical single patient records
- Segregating the queries across tables to eliminate long query processing times
- Grouping on patient ID and extracting unique values for each patient across every column using a set
- Storing the grouped data to S3 as a CSV file
- Combining the data across all tables to a single CSV file and storing it in S3

In [55]:
# Querying 4 tables patient, condition, immunization, and imagingstudy and storing it back on S3 as a CSV file

four_tables = pandas.read_sql("""SELECT patient.id as patientID
    , CONCAT(name[1].family, ' ', name[1].given[1]) as name
    , gender as gender
    , json_extract(patient.extension[1], '$.valueString') as MothersMaidenName
    , json_extract(patient.extension[2], '$.valueAddress.city') as birthPlace
    , maritalstatus.coding[1].display as maritalstatus
    , address[1].line[1] as addressline
    , address[1].city as city
    , address[1].district as district
    , address[1].state as state
    , address[1].postalcode as postalcode
    , address[1].country as country
    , json_extract(address[1].extension[1], '$.extension[0].valueDecimal') as latitude
    , json_extract(address[1].extension[1], '$.extension[1].valueDecimal') as longitude
    , telecom[1].value as telNumber
    , deceasedboolean as deceasedIndicator
    , deceaseddatetime
    , condition.id  as conditionId
    , condition.meta.tag[1].display
    , condition.encounter.reference as encounterId
    , condition.encounter.type as encountertype
    , json_extract(condition.modifierextension[1], '$.valueDecimal') AS confidenceScore
    , condition.category[1].coding[1].code as categoryCode
    , condition.category[1].coding[1].display as categoryDescription
    , condition.code.coding[1].code as diagnosisCode
    , condition.code.coding[1].display as diagnosisDescription
    , condition.onsetdatetime
    , severity.coding[1].code as severityCode
    , severity.coding[1].display as severityDescription
    , verificationstatus.coding[1].display as verificationStatus
    , clinicalstatus.coding[1].display as clinicalStatus
    , vaccinecode.text as vaccinations
    , series[1].modality.code as imageCode
    , series[1].modality.display as imagingName
    , series[1].bodySite.display as imageBodyPart
    , series[1].instance[1].title as imageTitle
FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.patient FULL JOIN
     multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.condition
     ON patient.id = condition.subject.reference FULL JOIN
     multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.immunization
     ON patient.id = immunization.patient.reference FULL JOIN
     multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.imagingstudy
     ON patient.id = imagingstudy.subject.reference
ORDER BY name;""", conn)

    
four_tables.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/four_tables.csv', index=False)
print("Complete")

Complete


In [13]:
# Aggregating data to include 1 record for every patient
# Creating a separate dataframe and storing it on S3 as a csv file
four_tables = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/four_tables.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
grouped_four_tables = four_tables.groupby(['patientID', 'name', 'gender', 'maritalstatus', 'addressline',
       'city', 'state', 'postalcode', 'country', 'latitude', 'longitude']).agg({'diagnosisDescription':set,
                                                                                'diagnosisCode':set,
                                                                                'onsetdatetime':set,
                                                                                'deceasedIndicator': set,
                                                                                'deceaseddatetime': set,
                                                                                'conditionId':set, 
                                                                                'categoryCode': set,
                                                                               'categoryDescription': set,
                                                                                'vaccinations':set, 
                                                                                'imageCode':set,
                                                                                'imagingName':set,
                                                                                'imageBodyPart':set,
                                                                                'imageTitle':set
                                                                               }).reset_index()
grouped_four_tables = grouped_four_tables.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_four_tables.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_four_tables.csv', index=False)
print("Complete")

Complete


In [7]:
# Creating a dataframe with the diagnosticreport table
diagnostic_report = pandas.read_sql("""SELECT  diagnosticreport.subject.reference as patientID, diagnosticreport.code.coding[1].display as diagnosticType
    , diagnosticreport.category[1].coding[1].display as diagnosticCategory
    , diagnosticreport.result[1].display as diagnosticResult
    FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.diagnosticreport""", conn)
diagnostic_report.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/diagnostic_report.csv', index=False)
print("Complete")

Complete


In [15]:
# diagnosticreport - Grouping the data based to a single record for each patient
grouped_diagnostic_report = diagnostic_report.groupby(['patientID']).agg({'diagnosticType':set,
                                                                                'diagnosticCategory':set,
                                                                                'diagnosticResult':set
                                                                               }).reset_index()
grouped_diagnostic_report = grouped_diagnostic_report.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_diagnostic_report.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_diagnostic_report.csv', index=False)
print("Complete")

Complete


In [16]:
#encounter- Creating a dataframe with the encounter table
encounter = pandas.read_sql("""SELECT encounter.subject.reference as patientID, encounter.class.code as encounterCode
    , encounter.reasoncode[1].coding[1].display as encounterReason
    , encounter.diagnosis as encounterDiagnosis
    , encounter.status as hospitalizationStatus
    FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.encounter""", conn)
encounter.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/encounter.csv', index=False)
print("Complete")

Complete


In [17]:
# encounter - Grouping the data based to a single record for each patient
grouped_encounter = encounter.groupby(['patientID']).agg({'encounterCode':set,
                                                                    'encounterReason':set,
                                                                    'encounterDiagnosis':set,
                                                                    'hospitalizationStatus': set,
                                                                   }).reset_index()
grouped_encounter = grouped_encounter.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_encounter.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_encounter.csv', index=False)
print("Complete")

Complete


In [18]:
# Creating a dataframe with the medicationadministration, and medicationrequest tables
medication = pandas.read_sql("""SELECT medicationrequest.subject.reference as patientID
    , medicationadministration.medicationcodeableconcept.coding[1].display as medicationDisplay
    , medicationadministration.effectivedatetime as medicationDate
    , medicationrequest.status as medicationStatus
    FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.medicationadministration FULL JOIN
    multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.medicationrequest
    ON medicationadministration.subject.reference = medicationrequest.subject.reference""", conn)
medication.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/medication.csv', index=False)
print("Complete")

Complete


In [19]:
# medicationadministration, and medicationrequest tables - Grouping the data based to a single record for each patient
grouped_medication = medication.groupby(['patientID']).agg({'medicationDisplay':set,
                                                                    'medicationDate':set,
                                                                    'medicationStatus': set
                                                                   }).reset_index()
grouped_medication = grouped_medication.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_medication.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_medication.csv', index=False)
print("Complete")

Complete


In [20]:
# claims - Creating a dataframe with the claim table
claims = pandas.read_sql("""SELECT claim.patient.reference as patientID, claim.use as claim
    FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.claim""", conn)
claims.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/claims.csv', index=False)
print("Complete")

Complete


In [21]:
# claims - Grouping the data based to a single record for each patient
grouped_claims = claims.groupby(['patientID']).agg({'claim':set}).reset_index()
grouped_claims = grouped_claims.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_claims.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_claims.csv', index=False)
print("Complete")

Complete


In [22]:
# observation - Creating a dataframe with the observation table
observation = pandas.read_sql("""SELECT observation.subject.reference as patientID
    , observation.code.text as observationType 
    , observation.valuecodeableconcept.text as observationText
    , observation.component[1].code.coding[1].display as observationDisplay
    FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.observation""", conn)
observation.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/observation.csv', index=False)
print("Complete")

Complete


In [23]:
# observation - Grouping the data based to a single record for each patient
grouped_observation = observation.groupby(['patientID']).agg({'observationType':set,
                                                                    'observationText':set,
                                                                    'observationDisplay': set
                                                                   }).reset_index()
grouped_observation = grouped_observation.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_observation.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_observation.csv', index=False)
print("Complete")

Complete


In [24]:
# Procedure - Creating a dataframe with the procedure table
procedure = pandas.read_sql("""SELECT procedure.subject.reference as patientID
    , procedure.code.text as procedureType
    FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.procedure""", conn)
procedure.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/procedure.csv', index=False)
print("Complete")

Complete


In [25]:
# Procedure Table - Grouping the data based to a single record for each patient
grouped_procedure = procedure.groupby(['patientID']).agg({'procedureType':set}).reset_index()
grouped_procedure = grouped_procedure.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_procedure.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_procedure.csv', index=False)
print("Complete")

Complete


In [26]:
# Reading all grouped tables
grouped_four_tables = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/grouped_four_tables.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
grouped_diagnostic_report = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/grouped_diagnostic_report.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
grouped_encounter = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/grouped_encounter.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
grouped_medication = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/grouped_medication.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
grouped_claims = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/grouped_claims.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
grouped_observation = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/grouped_observation.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
grouped_procedure = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/grouped_procedure.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])

In [32]:
grouped_all_clinical = grouped_diagnostic_report.merge(grouped_four_tables, how='left').merge(grouped_encounter).merge(grouped_medication).merge(grouped_claims).merge(grouped_observation).merge(grouped_procedure)
print("Totat number of patients with clinical records = ", len(grouped_all_clinical))
print(grouped_all_clinical.head(5))
grouped_all_clinical.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_all_clinical.csv', index=False)

Totat number of patients with clinical records =  1245
                              patientID  \
0  0074596f-5fd0-7965-db0f-cce71c81567d   
1  008d8e1d-c53f-47ac-fc06-aeb02ea5e3ec   
2  0122b006-83c5-b1fa-cb1d-a75934d9aef1   
3  013695bb-e8dd-a6cc-3bdf-14a1b4e107fb   
4  0159e45d-0e1c-4108-6cf5-0107b8825389   

                                      diagnosticType  \
0  'History and physical note', 'Comprehensive me...   
1  'History and physical note', 'Comprehensive me...   
2  'History and physical note', 'Lipid Panel', 'C...   
3  'History and physical note', 'Comprehensive me...   
4  'History and physical note', 'Genetic analysis...   

                          diagnosticCategory  \
0  'Laboratory', 'History and physical note'   
1  'Laboratory', 'History and physical note'   
2  'Laboratory', 'History and physical note'   
3  'Laboratory', 'History and physical note'   
4  'Laboratory', 'History and physical note'   

                                    diagnosticResult        

## Genomic single patient records

In [ ]:
# querying genomic data from Athena 

genomic_df = pandas.read_sql("""with    var as (select  sampleid,
                        contigname,
                        start,
                        referenceallele,
                        alternatealleles,
                        phased,
                        calls
                from "omicsdb"."synthea_newvariants_20230503t222941"),
                ann as (select  contigname,
                                start,
                                referenceallele,
                                alternatealleles,
                                attributes['GENEINFO'] as gene_info,
                                attributes['CLNSIG'] as clinical_significance,
                                phased,
                                calls
                        from "omicsdb"."synthea_annotations_20230503t222941"
                        where attributes['CLNDN'] is not null)
                SELECT  var.sampleid,
                        ann.gene_info,
                        ann.clinical_significance,
                        var.contigname,
                        var.start,
                        var.referenceallele,
                        var.alternatealleles,
                        var.phased,
                        var.calls
                FROM var inner join ann
                        -- genomic coordinates:
                        on var.contigname = ann.contigname
                        and var.start = ann.start

                        -- genomic sequence change:
                        and var.referenceallele = ann.referenceallele
                        and var.alternatealleles = ann.alternatealleles""", conn)
genomic_df.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/genomic_df.csv', index=False)
print("Complete")

Complete


In [35]:
genomic_df['gene_info'] = genomic_df['gene_info'].str.split(':').str[0]
genomic_df = genomic_df.rename(columns={"sampleid": "patientID"})
print(genomic_df.columns)

Index(['patientID', 'gene_info', 'clinical_significance', 'contigname',
       'start', 'referenceallele', 'alternatealleles', 'phased', 'calls'],
      dtype='object')


In [36]:
grouped_genomic_df = genomic_df.groupby(['patientID']).agg({'gene_info':set,
                                                                    'clinical_significance':set,
                                                                    'contigname': set,
                                                                    'start':set,
                                                                    'referenceallele':set,
                                                                    'alternatealleles': set,
                                                                   'phased':set,
                                                                    'calls':set
                                                                   }).reset_index()
grouped_genomic_df.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_genomic_df.csv', index=False)
print("Complete")

Complete


In [37]:
# Adding the diagnosis code and description columns to be able to identify patient't disease
grouped_all_genomic = grouped_all_clinical.merge(grouped_genomic_df, how='right')
grouped_all_genomic = grouped_all_genomic[['patientID', 'diagnosisDescription', 'diagnosisCode', 'gene_info',
       'clinical_significance', 'contigname', 'start', 'referenceallele',
       'alternatealleles', 'phased', 'calls']]
grouped_all_genomic = grouped_all_genomic.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_all_genomic.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_all_genomic.csv', index=False)

In [38]:
print('Totat number of patients with genomic records =', len(grouped_all_genomic))
print(grouped_all_genomic.head(5))

Totat number of patients with genomic records = 816
                              patientID  \
0  0074596f-5fd0-7965-db0f-cce71c81567d   
1  0159e45d-0e1c-4108-6cf5-0107b8825389   
2  0186c124-2b91-c2a7-79ea-c7fc40845943   
3  02281993-cc2a-9ee8-a652-91ecc0203e75   
4  0387ed06-9de8-e478-3269-c1ee7cd6d290   

                                diagnosisDescription  \
0                                                nan   
1                                                nan   
2                                                nan   
3  'Miscarriage in first trimester', 'Viral sinus...   
4  'Viral sinusitis (disorder)', 'Acute viral pha...   

                                       diagnosisCode  \
0                                                nan   
1                                                nan   
2                                                nan   
3  '429007001', '410429000', '19169002', '4448140...   
4  '10509002', '195662009', '55822004', '15777000...   

               

## Imaging single patient records

In [23]:
# updated imaging query

image_df = pandas.read_sql("""SELECT distinct patient.id, condition.code.coding[1].code as diagnosisCode, code.coding[1].display as diagnosisDescription, radiomicsfeatures.*
FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.patient,
     multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.condition,
     multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.imagingstudy,
     sagemaker_featurestore.imaging_feature_group_8bb9799a531d11ee99bbc1ec5fa87df4_1694709686 as radiomicsfeatures,
     coherent_dicom.studies
WHERE patient.id = condition.subject.reference
  and imagingstudy.identifier[1].value = CONCAT('urn:oid:',studies.studyinstanceuid)
  and imagingstudy.subject.reference = patient.id
  and studies.imagesetid = radiomicsfeatures.imagesetid""", conn)
image_df.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/image_df.csv', index=False)
print("Complete")

Complete


In [ ]:
# updated imaging query

image_df = pandas.read_sql("""SELECT distinct patient.id, condition.code.coding[1].code as diagnosisCode, code.coding[1].display as diagnosisDescription, radiomicsfeatures.*
FROM multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.patient,
     multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.condition,
     multimodalhclsdata_6e861ee724a35c9ab6b1ac0652c480b6_healthlake_view.imagingstudy,
     sagemaker_featurestore.imaging_feature_group_8bb9799a531d11ee99bbc1ec5fa87df4_1694709686 as radiomicsfeatures,
     coherent_dicom.studies
WHERE patient.id = condition.subject.reference
  and imagingstudy.identifier[1].value = CONCAT('urn:oid:',studies.studyinstanceuid)
  and imagingstudy.subject.reference = patient.id""", conn)
image_df.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/image_df.csv', index=False)
print("Complete")

In [24]:
image_df = image_df.rename(columns={"id": "patientID"})
columns = image_df.columns.tolist()
columns = columns[1:]
grouped_all_imaging= image_df.groupby('patientID').agg(lambda x: set(x)).reset_index()
grouped_all_imaging = grouped_all_imaging.astype(str).replace({"{":"", "}":""}, regex=True)
grouped_all_imaging = grouped_all_imaging.loc[:, ~grouped_all_imaging.columns.str.startswith('diagnostics')]
grouped_all_imaging.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_all_imaging.csv', index=False)

In [25]:
print('Totat number of patients with imaging records =', len(grouped_all_imaging))
print(grouped_all_imaging.head(5))

Totat number of patients with imaging records = 298
                              patientID  \
0  0074596f-5fd0-7965-db0f-cce71c81567d   
1  0122b006-83c5-b1fa-cb1d-a75934d9aef1   
2  022fe2f8-95dd-54c8-7d0a-f05d5a74d39e   
3  02551c0a-12e2-a8d0-d756-408440072685   
4  031a8ae5-e01e-bcc9-f83f-874cb909b641   

                                       diagnosisCode  \
0  '233604007', '162864005', '22325002', '3868060...   
1  '233604007', '162864005', '22325002', '3868060...   
2  '22325002', '410429000', '309557009', '7238570...   
3  '22325002', '271737000', '70704007', '49436004...   
4  '386806002', '22325002', '10509002', '28454900...   

                                diagnosisDescription  \
0  'Impaired cognition (finding)', 'Hyperlipidemi...   
1  'Impaired cognition (finding)', 'Prediabetes',...   
2  'Abnormal gait (finding)', 'Viral sinusitis (d...   
3  'Hyperlipidemia', "Alzheimer's disease (disord...   
4  'Impaired cognition (finding)', 'Prediabetes',...   

  original_shap

## Extracting patients that have all 3 modalities

In [26]:
clinical = pandas.read_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_all_clinical.csv', encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
genomic = pandas.read_csv("s3://multimodal-dataset-clinical-genomic-imaging/grouped_all_genomic.csv", encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])
image = pandas.read_csv('s3://multimodal-dataset-clinical-genomic-imaging/grouped_all_imaging.csv', encoding='utf-8', dtype='str', keep_default_na=False, na_values=[''])

In [27]:
!pip install pandasql
from pandasql import sqldf

  Using cached pandasql-0.7.3-py3-none-any.whl

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [47]:
clinical_query = '''select clinical.patientID, diagnosticType, diagnosticCategory, diagnosticResult,
       name, gender, maritalstatus, addressline, city, state,
       postalcode, country, latitude, longitude,
       clinical.diagnosisDescription, clinical.diagnosisCode, onsetdatetime,
       deceasedIndicator, deceaseddatetime, conditionId, categoryCode,
       categoryDescription, vaccinations, imageCode, imagingName,
       imageBodyPart, imageTitle, encounterCode, encounterReason,
       encounterDiagnosis, hospitalizationStatus, medicationDisplay,
       medicationDate, medicationStatus, claim, observationType,
       observationText, observationDisplay, procedureType
       from clinical, genomic, image where clinical.patientID = genomic.patientID AND clinical.patientID = image.patientID'''
final_clinical_df = sqldf(clinical_query)
print(final_clinical_df.columns)
print(len(final_clinical_df))
final_clinical_df.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/final_clinical_df.csv', index=False)

Index(['patientID', 'diagnosticType', 'diagnosticCategory', 'diagnosticResult',
       'name', 'gender', 'maritalstatus', 'addressline', 'city', 'state',
       'postalcode', 'country', 'latitude', 'longitude',
       'diagnosisDescription', 'diagnosisCode', 'onsetdatetime',
       'deceasedIndicator', 'deceaseddatetime', 'conditionId', 'categoryCode',
       'categoryDescription', 'vaccinations', 'imageCode', 'imagingName',
       'imageBodyPart', 'imageTitle', 'encounterCode', 'encounterReason',
       'encounterDiagnosis', 'hospitalizationStatus', 'medicationDisplay',
       'medicationDate', 'medicationStatus', 'claim', 'observationType',
       'observationText', 'observationDisplay', 'procedureType'],
      dtype='object')
151


In [48]:
genomic_query = '''select clinical.patientID, clinical.diagnosisDescription, clinical.diagnosisCode, gene_info, clinical_significance, contigname,
       start, referenceallele, alternatealleles, phased, calls
       from clinical, genomic, image where clinical.patientID = genomic.patientID AND clinical.patientID = image.patientID'''
final_genomic_df = sqldf(genomic_query)
print(final_genomic_df.columns)
print(len(final_genomic_df))
final_genomic_df.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/final_genomic_df.csv', index=False)

Index(['patientID', 'diagnosisDescription', 'diagnosisCode', 'gene_info',
       'clinical_significance', 'contigname', 'start', 'referenceallele',
       'alternatealleles', 'phased', 'calls'],
      dtype='object')
151


In [28]:
imaging_query = '''select * from clinical, genomic, image where clinical.patientID = genomic.patientID AND clinical.patientID = image.patientID'''
image_df = sqldf(imaging_query)
image_df = image_df.drop(columns = ['diagnosticType', 'diagnosticCategory', 'diagnosticResult',
       'name', 'gender', 'maritalstatus', 'addressline', 'city', 'state',
       'postalcode', 'country', 'latitude', 'longitude', 'onsetdatetime',
       'deceasedIndicator', 'deceaseddatetime', 'conditionId', 'categoryCode',
       'categoryDescription', 'vaccinations', 'imageCode', 'imagingName',
       'imageBodyPart', 'imageTitle', 'encounterCode', 'encounterReason',
       'encounterDiagnosis', 'hospitalizationStatus', 'medicationDisplay',
       'medicationDate', 'medicationStatus', 'claim', 'observationType',
       'observationText', 'observationDisplay', 'procedureType', 'gene_info',
       'clinical_significance', 'contigname', 'start', 'referenceallele',
       'alternatealleles', 'phased', 'calls', 'eventtime', 'write_time',
       'api_invocation_time', 'is_deleted'])
final_imaging_df =image_df.iloc[:,~image_df.columns.duplicated()]
final_imaging_df.to_csv('s3://multimodal-dataset-clinical-genomic-imaging/final_imaging_df.csv', index=False)
print(final_imaging_df.columns)
print(len(final_imaging_df))

Index(['patientID', 'diagnosisDescription', 'diagnosisCode',
       'original_shape_elongation', 'original_shape_flatness',
       'original_shape_leastaxislength', 'original_shape_majoraxislength',
       'original_shape_maximum2ddiametercolumn',
       'original_shape_maximum2ddiameterrow',
       'original_shape_maximum2ddiameterslice',
       ...
       'original_glszm_zoneentropy', 'original_glszm_zonepercentage',
       'original_glszm_zonevariance', 'original_ngtdm_busyness',
       'original_ngtdm_coarseness', 'original_ngtdm_complexity',
       'original_ngtdm_contrast', 'original_ngtdm_strength',
       'imagingstudyidvalue', 'imagesetid'],
      dtype='object', length=112)
151
